# 연습문제 1

- 3인 1조로 조를 구성하시오
- 감성대화말뭉치_validation.csv의 여러 컬럼 중 [사람문장 1] 컬럼만 엑셀로 가져오시오
- 가져온 컬럼 중 600행을 가져오시오 (조별로 가져오는 행을 다르게 함)
    - 1. 조원끼리 분담하여 600행의 데이터에 대하여 긍정/중립/부정 라벨링을 하시오 (감정 분류 내용을 따르지 않고, 다시 라벨링을 진행. 중립은 많지 않음)
    - 2. 600행 데이터에서 긍정 단어와 부정 단어를 찾아 조별 토의로 감정사전을 만드시오 
    - 3. 조별로만든감정사전을통합하되,같은단어가있는경우자기조의판단내용을유지

In [9]:
# import numpy as np
# import pandas as pd

# path = '/Users/jsha/gjai/nlp/pytest/'
# file = '감성대화말뭉치(최종데이터)_Validation.xlsx'

# df = pd.read_excel(path+file)
# df2 = df.loc[1651:1800, ['감정_대분류', '감정_소분류', '사람문장1']]
# df2.to_excel(path+'jsha_감성분석.xlsx', index=False)

In [33]:
import numpy as np
import pandas as pd

path = '/Users/jsha/gjai/nlp/pytest/'
file2 = 'jsha_감성분석_labeled.xlsx'

df = pd.read_excel(path+file2)
df = df.iloc[:, [0,1,2,-1]]
df.rename(columns={'Unnamed: 12':'label'}, inplace=True)
df[['사람문장1', 'label']].to_csv(path+'labeling_1651_1800.csv', encoding='cp949', index=False)

In [116]:
file1 = 'labeling_1201_1350.csv'
file2 = 'labeling_1351_1500.xlsx'
file3 = 'labeling_1501_1650.txt'
file4 = 'labeling_1651_1800.csv'

df1 = pd.read_csv(path+file1)
df2 = pd.read_excel(path+file2)
with open(path+file3, 'r', encoding='cp949') as f:
    data = f.readlines()
    data = [da.split('\t') for da in data]
    data = data[1:]
dummy = [(ll[1], ll[2].replace('\n', '')) for ll in data]
df3 = pd.DataFrame(dummy, columns=['text', 'label'])
df4 = pd.read_csv(path+file4, encoding='cp949')
df1.columns = ['text', 'label']
df2.columns = ['text', 'label']
df3.columns = ['text', 'label']
df4.columns = ['text', 'label']

In [124]:
new = pd.concat([df1, df2, df3, df4], axis=0)
new['label'] = pd.to_numeric(new['label'])
new.info()
new.to_csv(path+'통합라벨데이터.csv', encoding='cp949', index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600 entries, 0 to 149
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    600 non-null    object
 1   label   600 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 14.1+ KB


In [43]:
import pandas as pd
import numpy as np

path = '/Users/jsha/gjai/nlp/pytest/'
file2 = '통합라벨링데이터.csv'

df = pd.read_csv(path+file2)
df.shape

(600, 2)

In [ ]:
import rhinoMorph

rn = rhinoMorph.startRhino()

def morph(string):
    ll = rhinoMorph.onlyMorph_list(rn, string,
                pos= ['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 
                      'MM', 'MAG', 'MAJ'], eomi=True, xrVv=True)
    return ll

df['morphed'] = df['text'].apply(morph)
df.head()

## 긍정 감정사전 만들기

In [53]:
positive = df[['text', 'morphed']][df['label']==1]
new_positive = positive.sum()
new_positive
# print(len(new_positive))
# new_positive = list(set(new_positive))
# print(len(new_positive))
# pd.Series(new_positive).to_excel(path+'positive.xlsx', index=False)

text       요즈음 사는 기분이 나.팀원들의 업무분배로 인한 오해가 풀렸어.나 내일 결혼해서 너...
morphed    [요즈음, 살다, 기분, 나다, 팀원, 업무, 분배, 인하다, 오해, 풀리다, 내일...
dtype: object

In [68]:
positive_done_df = pd.read_excel(path+'positive_done.xlsx')

positive_done_df.columns = ['a', 'positive_word']
positive_done = positive_done_df['positive_word']
positive_done.dropna(inplace=True)

In [70]:
positive_done.reset_index(drop=True)

0        기증자
1         사다
2         돕다
3       배부르다
4         맡다
       ...  
150       적금
151      밤하늘
152    만족스럽다
153       찾다
154      다행히
Name: positive_word, Length: 155, dtype: object

In [89]:
positive_done.to_csv(path+'positive_dict_jsha.csv', index=False)

## 부정 감정사전 만들기

In [81]:
negative = df['morphed'][df['label']==-1]
new_negative = negative.sum()
new_negative
print(len(new_negative))
new_negative = list(set(new_negative))
print(len(new_negative))
pd.Series(new_negative).to_excel(path+'negative.xlsx', index=False)

3784
1208


In [87]:
negative_done = pd.read_excel(path+'negative_done.xlsx')
negative_done.columns = ['negative_word']
negative_done = negative_done['negative_word']
negative_done.dropna(inplace=True)

In [88]:
negative_done.reset_index(drop=True)

0       우울하다
1        조르다
2      부담스럽다
3       파기하다
4      잃어버리다
       ...  
487     싫어하다
488     쓸쓸하다
489        약
490       아무
491       가시
Name: negative_word, Length: 492, dtype: object

In [90]:
negative_done.to_csv(path+'negative_dict_jsha.csv', index=False)

## 중립 감정사전 만들기

In [91]:
neural = df['morphed'][df['label']==0]
new_neural = neural.sum()
new_neural
print(len(new_neural))
new_neural = list(set(new_neural))
print(len(new_neural))
pd.Series(new_neural).to_excel(path+'neural.xlsx', index=False)

222
171


In [100]:
neural_done = pd.read_excel(path+'neural_done.xlsx')
type(neural_done)
neural_done = neural_done['neutral_word']
neural_done.dropna(inplace=True)

In [101]:
neural_done.reset_index(drop=True)

0      따르다
1      대하다
2       사다
3      맞추다
4      챙기다
5       다음
6      적당히
7     익숙하다
8     분배하다
9       살다
10     다니다
11       밥
12    요청하다
13      매일
14    움직이다
15      입다
16     어리다
17       안
18      어제
19       일
20      남다
21      수업
22      주변
23      면접
24    관련하다
25      도움
26     채우다
27    나타나다
28      과장
29      먹다
30     보이다
31      주다
32      있다
33      자다
34      연락
35      회사
36     나누다
37      요즘
38    늘어나다
39      필요
40    담당하다
41       애
42      의견
43      가다
44      교복
45      시선
46      마음
47     앞두다
48      전체
49    챙겨주다
50      되다
51      평소
52     나오다
Name: neutral_word, dtype: object

In [102]:
neural_done.to_csv(path+'neutral_dict_jsha.csv', index=False)